In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb

# Tokenizer 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

# 질의를 임베딩으로 변환하는 함수 (mean pooling 포함)
def tokenize_and_pooling(model, tokenizer, text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    attention_mask = encoded_input['attention_mask']
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

c:\Users\MoonDongWon\OneDrive - gnu.ac.kr\바탕 화면\프로젝트\과제 도우미\report_helper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\MoonDongWon\OneDrive - gnu.ac.kr\바탕 화면\프로젝트\과제 도우미\report_helper\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
client = chromadb.PersistentClient('../../chroma')

collection_name = "paper_embeddings"
try:
    paper_embeddings = client.get_collection(name=collection_name)
except:
    paper_embeddings = client.create_collection(name=collection_name)

In [3]:
def search(query: str):
    query_text = query
    query_embedding = tokenize_and_pooling(model, tokenizer, query_text)
    results = paper_embeddings.query(
        query_embeddings=query_embedding.numpy().tolist(),
        n_results=5
    )
    for result in results['metadatas'][0]:
        print(result['title'], '\n')

In [6]:
search('감성을 이해할 수 있을까?')

CNN-LSTM 조합모델을 이용한 영화리뷰 감성분석 

집단지성을 이용한 한글 감성어 사전 구축 

국내 주요 10대 기업에 대한 국민 감성 분석 

영화 리뷰 감성분석을 위한 텍스트 마이닝 기반 감성 분류기 구축 

감성 평가를 위한 감성의 의미 재정립과 어휘 체계에 관한 연구 

